## Purpose
This notebook is for testing the pbf conversion tool to implement different speeds on the OSM ways.

In [3]:
import xml.etree.ElementTree as ET
import pandas as pd
import gc
from timeit import default_timer as timer
import numpy as np

In [2]:
road_path = '/Users/askalila/Documents/TfC Offline Work/numo/Network/Clutter/trial_3sf.osm'


In [462]:
tree = ET.parse(road_path)

In [463]:
root = tree.getroot()

### Test 1
Change maxspeed on all ways in the osm file to 200 mph

In [418]:
number_ofways = len(root.findall('./way'))
print('Total number of ways in the osm:', number_ofways)

Total number of ways in the osm: 165984


Steps to follow for test 1
1. iterate over all ways
2. if there is a node named tag with attribute key maxspeed, change its value to 200
3. if there is no node tag with attribute maxspeed, create such a tag and assign it the value of 200 mph

In [111]:
set_static_speed = 200 # in mph
speedStr = str(set_static_speed) +' mph'

for way in root.findall('way'):
    if way.find('tag/[@k="maxspeed"]') is None: # if there is no maxpseed tag
        ET.SubElement(way, 'tag')   # adds a tag element under way
        way.findall(".//tag[last()]")[0].set('k', 'maxspeed') # adds attribute key as 'maxspeed'
        way.findall(".//tag[last()]")[0].set('v', speedStr) # adds attribute value as static Speed above
    else:
        way.find('tag/[@k="maxspeed"]').attrib['v'] = speedStr
            

In [115]:
# output it to .osm file
tree.write('200sf3.osm', encoding="utf-8",xml_declaration=True)

### Test 2

Steps to follow for test 2:
1. read entire speed csv and drop all rows not 8 am
2. iterate through the rows of the speeds df and 

In [1]:
filename = '/Users/askalila/Documents/TfC Offline Work/numo/Uber Speed/movement-speeds-quarterly-by-hod-san-francisco-2020-Q1.csv'
filename8am = '/Users/askalila/Documents/TfC Offline Work/numo/Uber Speed/movement-speeds-quarterly-by-hod-san-francisco-2020-Q1_8am.csv'

In [4]:
# speedsDF8am = speedsAll[speedsAll.hour_of_day == 8]
# speedsDF8am = pd.read_csv(filename8am)
# speedsDFpeak = speedsDF8am
speedsDF = pd.read_csv(filename)

In [464]:
## Run the functions on all the ways in the osm xml tree

newwayID_seed = 1000  # This is a potential bug if wayID


start = timer()
# ...


for i,originalway in enumerate(root.findall("./way")):
    wayID = originalway.attrib['id']
    CheckNodesForSpeeds(wayID, originalway, 'mph')
    
    
    if i in [1660,16600,83000,160000]: 
        bookmark = timer()
        progress = int(i/(number_ofways-1)*100)
        print(f'{progress}% completed in {np.round((bookmark - start)/60)} minutes...')

        
    # delete original ways with ref nodes 1 or fewer
    if len(originalway.findall("./nd")) < 2: root.remove(originalway)
        
end = timer()
print(f'100% completed in {np.round((end- start)/60)} minutes!')

-31123.933517542
0% completed in 0.0 minutes...
9% completed in 3.0 minutes...
49% completed in 25.0 minutes...
96% completed in 40.0 minutes...
100% completed in 40.0 minutes!


In [465]:
# output it to .osm file
tree.write('RealspeedSF1.osm', encoding="utf-8",xml_declaration=True)

In [12]:
del speedsAll


NameError: name 'speedsAll' is not defined

In [13]:
gc.collect()

17

In [182]:
speedsDF8am.to_csv('/Users/askalila/Documents/TfC Offline Work/numo/Uber Speed/movement-speeds-quarterly-by-hod-san-francisco-2020-Q1_8am.csv')



In [183]:
print('No. of unique ways:',len(set(speedsDF8am.osm_way_id)))
print('No. of unique start nodes',len(set(speedsDF8am.osm_start_node_id)))

No. of unique ways: 82660
No. of unique start nodes 153832


In [135]:
len(root.findall('.//node'))

824582

## Scratch Pad Below 

In [160]:
wayexample = speedsDF8am.iloc[10].osm_way_id
startnode = speedsDF8am.iloc[10].osm_start_node_id
endnode = speedsDF8am.iloc[10].osm_end_node_id

In [156]:
buildstr = "./way/[@id =\"" + str(wayexample) + "\"]"
way3 = root.findall(buildstr)


In [ ]:
buildstr = "./way/[@id =\"" + str(wayexample) + "\"]"

In [9]:
ways4 = root.findall('./way/nd/[@ref ="65365166"]..')

In [163]:
for node in root.findall('./way/nd/..[@ref ="65365166"]'):
    print(node.attrib)

In [274]:
# test here
way1 = root.find('./way/[@id ="476005551"]') # does not have maxspeed
way2 = root.find('./way/[@id ="476007033"]') # has maxspeed attr in tag

# Example with several nodes in the nodepair with real speed:
wayID = "27028340" 
way5 = root.find("./way/[@id = '27028340']")

In [33]:
for node in way1.iter('tag/[@k="highway"]'):
    
    print(node)

In [291]:
for node in root.findall("./way/[@id = '11002']"):
    ET.dump(node)
    print('next')

<way id="11002" version="11" timestamp="2019-03-19T05:40:02Z" changeset="68281580" uid="119881" user="clay_c" synthetic="yes"><nd ref="65296374" /><nd ref="65306688" /><tag k="highway" v="residential" />
		<tag k="name" v="29th Street" />
		<tag k="tiger:cfcc" v="A41" />
		<tag k="tiger:county" v="San Francisco, CA" />
		<tag k="tiger:name_base" v="29th" />
		<tag k="tiger:name_type" v="St" />
		<tag k="tiger:reviewed" v="no" />
	<tag k="maxspeed" v="18.32" /></way>
next


In [174]:
if way1.find('tag/[@k="maxspeed"]') is None:
    print("It is in fact a None")
else:
    print("No, it is Not None")

No, it is Not None


In [138]:
speedsDF[speedsDF.hour_of_day == 8]

,year,quarter,hour_of_day,segment_id,start_junction_id,end_junction_id,osm_way_id,osm_start_node_id,osm_end_node_id,speed_mph_mean,speed_mph_stddev,speed_mph_p50,speed_mph_p85
25,2020,1,8,f32dbf217023581f429d56330be2a16410bc2809,8aaf6ad421333ad741cb1d5de1e3fa83e7d0e908,8bbd97259361a23d374e60e6582019550fc58e0f,417094233,4714793573,1014244233,28.066,5.100,29.005,32.166
55,2020,1,8,aa884b48dd741bb591001ed9d77aaac9f4466048,7fe20e98f1cf38b3825fb4cb5bf10a7088834462,8bbd97259361a23d374e60e6582019550fc58e0f,87223163,1014244242,1014244233,24.026,2.734,24.095,26.280
70,2020,1,8,b51e0d015f2a7d9a417746aaae39bbdbfd47a6be,7c4361ada161cd0725a010cdbd44d0213a6f9efc,492441475679f95357fbeefc00a9e49c544081c0,416137931,4169719264,4318478540,28.965,8.187,27.997,31.450
79,2020,1,8,9d2084bc7090e387df9866d22b383f2537c40628,e08d4f15ac0eb1ce777ea2d54efaf6ffa2c3c30e,492441475679f95357fbeefc00a9e49c544081c0,416137931,239464357,4318478540,26.633,4.155,26.655,30.722
94,2020,1,8,d4271970a0d9f3db34b3f78e4687e7fccc22e164,1cfca404596ca398165e882e27eb1a78e7676a10,6fb71af79d00f26437027d1cc9942e684c60f5e7,8725228,62444228,62478010,30.484,5.780,29.999,34.574
...,...,...,...,...,...,...,...,...,...,...,...,...,...
499970,2020,1,8,44831a9500b97f34adcb9886db35dd15493967a2,7b4c260e02642a20536eb7f27bbbf5d5b24c4221,1eeb9662402e24b0518561956d283f0f2991ed1b,8933649,65440060,5437711919,19.475,6.815,19.452,25.191
499971,2020,1,8,5d4cfde6346f43c15b95eb6ec1c63394063beca9,a77097ab77d9a95f54e24c8c838c89d513c72d0e,b1ef4d473b7bdc6dc48fdb84819eb5a657ef13d5,7702531,55958732,4926336868,21.133,2.786,21.067,23.264
499972,2020,1,8,42ffdb101280ebd7f643007ad86053b0e7c87f9c,75a3a751901afde7178596c2f33b183d518d84fb,fed0fdba7108a62773efd32da7d7f89bcf4f8e14,7713726,56012896,56093652,35.578,4.569,35.245,38.873
499987,2020,1,8,2675e49fef53ba6b2601664e4ad38658690bc360,ec5645150a6767900fc26a45cea3aa1c13fa2a08,a4f06693b1460b9eda9f667eca48a1296fff07bb,396980342,65308402,65309302,21.684,4.530,21.893,25.082


- iterate through all ways (try first with just one)
- create a list of the nodes within using findall 
- pass to function to do: in speedsDF,  query for nodelist in start node and end node. returns rows with relevant speeds in either direction
    - if list > 0, take the resulting node paris of each row and create ways with the pairs (loop)
            then take all of the nd's from i to the to_node's position in the way
        create a new way in the tree and append those nodes, copy over all tags and add a maxpeed tag
        remove the nd's from the original way
    - if list is empty, move to next way

- repeat the above until either (only one nd in the way) or (nd's do not return a speed in df)
- add an attribute to the way to indicate that it has been checked for speed
- in tree, remove ways with only nd's
    


theres a speed


In [453]:
def CopyNodes (originalway, nodesequence, realspeed, units, newwayID, verbose = 'no'):
    """
    Purpose: this function creates a new way with the nodes that have speed and adds attribute to the old and new ways to mark them as synthetic
    
    input:
        originalway: original way element object
        nodesequence: list of node IDs to be copied as strings in list, in the correct direction
        realspeed: mean speed as string or 'no' to skip changing maxspeed tag
        units: string of 'mph' or'kph'
        newwayID: string with a new way ID number
        
    no outputs
    """

    newway = ET.Element('way', attrib = dict(originalway.items())) #creates new way with same attributes

    # add nodes as nd children
    
    for i,anode in enumerate(nodesequence):    
        NodeRef = ET.Element('nd', attrib = {'ref':anode})
        newway.append(NodeRef)
        

    # adds attributes and tags to new way
    newway.set('id', newwayID) #replace orig way id with new id
    newway.set('synthetic','yes')
    newway.extend(originalway.findall("./tag")) #copies over all the exisitng tags

    if realspeed != 'no': 
    
        # add the maxspeed tag from real speed or replace current maxspeed with real speed
        if newway.find('tag/[@k="maxspeed"]') is None:
            speedtag = ET.Element('tag', attrib = {'k':'maxspeed','v':realspeed})
            newway.append(speedtag)   # adds a tag element under way
        else:
            newway.find('tag/[@k="maxspeed"]').attrib['v'] = realspeed

    
    global root
    root.append(newway) # adds new way to the xml
    if verbose == 'yes': print('adding new way with ID',newwayID)
    
    
    

In [5]:

speedsDF


,year,quarter,hour_of_day,segment_id,start_junction_id,end_junction_id,osm_way_id,osm_start_node_id,osm_end_node_id,speed_mph_mean,speed_mph_stddev,speed_mph_p50,speed_mph_p85
0,2020,1,0,8f4827ebed3c2e66f50daef967d5e91daadd8d98,8e555723c3dff79036c7a8c0cef6b32a80763c9f,2278ad9374ec96c35a0d769bc8a275f6355b55da,40722998,62385707,4927951349,26.078,3.658,25.724,28.917
1,2020,1,10,8f4827ebed3c2e66f50daef967d5e91daadd8d98,8e555723c3dff79036c7a8c0cef6b32a80763c9f,2278ad9374ec96c35a0d769bc8a275f6355b55da,40722998,62385707,4927951349,22.416,4.212,22.886,26.041
2,2020,1,15,8f4827ebed3c2e66f50daef967d5e91daadd8d98,8e555723c3dff79036c7a8c0cef6b32a80763c9f,2278ad9374ec96c35a0d769bc8a275f6355b55da,40722998,62385707,4927951349,20.154,4.290,20.200,24.310
3,2020,1,19,8f4827ebed3c2e66f50daef967d5e91daadd8d98,8e555723c3dff79036c7a8c0cef6b32a80763c9f,2278ad9374ec96c35a0d769bc8a275f6355b55da,40722998,62385707,4927951349,22.133,3.594,22.386,25.697
4,2020,1,1,8f4827ebed3c2e66f50daef967d5e91daadd8d98,8e555723c3dff79036c7a8c0cef6b32a80763c9f,2278ad9374ec96c35a0d769bc8a275f6355b55da,40722998,62385707,4927951349,27.004,4.085,26.585,30.230
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4900802,2020,1,11,44efca99fd097a0934504a8402b89a6ea8c1665f,3e63162120e16ba509c6cde091ba48b530b46074,5e68db681c845bb5a536992e550a68d7d40ca116,416139364,4169729066,575497818,11.741,12.108,5.601,24.784
4900803,2020,1,18,44efca99fd097a0934504a8402b89a6ea8c1665f,3e63162120e16ba509c6cde091ba48b530b46074,5e68db681c845bb5a536992e550a68d7d40ca116,416139364,4169729066,575497818,8.925,9.643,4.822,22.537
4900804,2020,1,6,44efca99fd097a0934504a8402b89a6ea8c1665f,3e63162120e16ba509c6cde091ba48b530b46074,5e68db681c845bb5a536992e550a68d7d40ca116,416139364,4169729066,575497818,14.950,11.632,14.182,29.022
4900805,2020,1,19,44efca99fd097a0934504a8402b89a6ea8c1665f,3e63162120e16ba509c6cde091ba48b530b46074,5e68db681c845bb5a536992e550a68d7d40ca116,416139364,4169729066,575497818,7.827,9.308,3.248,18.131


In [446]:

    # Examples of different ways
# wayID = "400104475"  #example with all nodes have 6 different real speed segments
wayID = "404848898"
# wayID = "1030325062" 
# wayID = "6354674"

originalway = root.find(".way/[@id = '" +wayID+ "']")

# newwayID_seed = 12350



In [447]:
#before
ET.dump(originalway)

<way id="404848898" version="9" timestamp="2021-05-11T09:14:46Z" changeset="104500543" uid="14293" user="KindredCoda">
		<nd ref="4177129055" />
		<nd ref="4071044664" />
		<nd ref="8721081403" />
		<nd ref="4071045510" />
		<nd ref="8721081404" />
		<nd ref="4071045529" />
		<nd ref="4071044665" />
		<nd ref="8721081401" />
		<nd ref="4071045509" />
		<nd ref="4071044682" />
		<nd ref="4071045507" />
		<nd ref="8721081402" />
		<nd ref="4177129056" />
		<tag k="highway" v="secondary" />
		<tag k="lanes" v="4" />
		<tag k="lcn_ref" v="885" />
		<tag k="maxspeed" v="40 mph" />
		<tag k="name" v="Lake Merced Boulevard" />
		<tag k="tiger:cfcc" v="A45" />
		<tag k="tiger:county" v="San Francisco, CA" />
		<tag k="tiger:name_base" v="Lake Merced" />
		<tag k="tiger:name_type" v="Blvd" />
	</way>
	


In [448]:
CheckNodesForSpeeds(wayID, originalway, 'mph', verbose='yes')

adding new way with ID 1001
adding new way with ID 1002
removed node index 0
removed node index 1
removed node index 2
removed node index 3
removed node index 4
removed node index 5
removed node index 6
removed node index 7
removed node index 8
removed node index 9
removed node index 10
removed node index 11
copy pairs: []


In [449]:
#after
ET.dump(originalway)

print(len(originalway.findall("./nd")))


<way id="404848898" version="9" timestamp="2021-05-11T09:14:46Z" changeset="104500543" uid="14293" user="KindredCoda">
		<nd ref="4177129056" />
		<tag k="highway" v="secondary" />
		<tag k="lanes" v="4" />
		<tag k="lcn_ref" v="885" />
		<tag k="maxspeed" v="34.165" />
		<tag k="name" v="Lake Merced Boulevard" />
		<tag k="tiger:cfcc" v="A45" />
		<tag k="tiger:county" v="San Francisco, CA" />
		<tag k="tiger:name_base" v="Lake Merced" />
		<tag k="tiger:name_type" v="Blvd" />
	</way>
	
1


In [451]:
root.remove(originalway)
originalway = root.find(".way/[@id = '" +wayID+ "']")
ET.dump(originalway)

ValueError: list.remove(x): x not in list

In [454]:
def CheckNodesForSpeeds (wayID, originalway, units, verbose = 'no'):
    """
    This function will check the Speeds Dataframe (global variable) for speeds between node pairs in the way.
    
    Global variables that need to be present:
        root
        speedsDFpeak
    inputs: 
        wayID: original way element's ID as string
        nodelist: the way's list of node reference ids as strings
        units: 'mph' or 'kph' string
        newwayID_seed: starting number for new way's to iterate over by 1
    
    function Calls another function to do:
        - creates new way with nodes and speed and copies all metadata and non-speed tags
        - removes the copied nodes from the original way
        - adds attribute to newway "synthetic = yes"
        
        - adds attribute to origin way "Checked_for_speed = yes"
        
    no outputs
    """
    global newwayID_seed

    #create nodelist of node id strings and similar list of floats
    nodelist = []
    nodelistfloat = []
    for node in originalway.iter('nd'):
        
        ref = node.attrib['ref']
        nodelist.append(ref)
        nodelistfloat.append(int(ref))

    # get reverse of nodelist
    reverseNodelistfloat = list(reversed(nodelistfloat))
    reverseNodelist = list(reversed(nodelist))

    # get only speeds with origin AND Destination nodes in the original way
    relevantSpeeds = speedsDFpeak.loc[(speedsDFpeak.osm_start_node_id.isin(nodelistfloat)) & (speedsDFpeak.osm_end_node_id.isin(nodelistfloat))]

    # nodepairtuples = []
    nodepairdict = dict()
    # create new ways from the relevant speeds
    for nodes in relevantSpeeds[['osm_start_node_id','osm_end_node_id','speed_mph_mean']].iterrows():
        pair = list(nodes[1])
        startnode, endnode, realspeed_float = pair[0],pair[1], pair[2]
        realspeed = str(realspeed_float)
        newwayID = str(newwayID_float)

        # check if its the direction of node sequence or reverse
        if nodelistfloat.index(endnode) > nodelistfloat.index(startnode): #correct sequence     
            startIndex, endIndex = nodelistfloat.index(startnode), nodelistfloat.index(endnode)
            CopyNodes(originalway, nodelist[startIndex:endIndex+1],realspeed,'mph',str(newwayID_seed))
            newwayID_seed += 1
    #         nodepairtuples.append((startIndex, endIndex))  # no longer needed
            nodepairdict[startIndex] = endIndex

        elif nodelistfloat.index(endnode) < nodelistfloat.index(startnode): # reverse
            # check if the reverse nodepair belongs to another way (from DF in Uber, from xml tree in mapbox), 
            # if yes then skip this block
            try: # check if the speed DF has a different way ID for the reverse node direction (n/a to mapbox speed)
                samewayID = wayID == str(list(speedsDFpeak.loc[(speedsDFpeak.osm_start_node_id == startnode) & (speedsDFpeak.osm_end_node_id == endnode)]['osm_way_id'])[0])
            except: # for mapbox, check the root for the node pair's ways [incomplete!!]
                samewayID = True
                pass
            
            if samewayID:
                startIndex, endIndex = nodelistfloat.index(startnode), nodelistfloat.index(endnode)
                revstartIndex, revendIndex = reverseNodelistfloat.index(startnode), reverseNodelistfloat.index(endnode)
                CopyNodes(originalway, reverseNodelist[revstartIndex:revendIndex+1],realspeed,'mph',str(newwayID_seed))
                newwayID_seed += 1
        #         nodepairtuples.append((startIndex, endIndex))  # no longer needed

        else: raise Exception('start node or end node not in way nodelist')


    ## remove nodes from original way

    startPos = 0
    endPos = len(nodelist)-1

    data_items = nodepairdict.items()
    data_list = list(data_items)
    nodepairdf = pd.DataFrame(data_list,columns=['start','end'])

    # first, delete nodes from segemnts at the start of the way

    while len(nodepairdf[nodepairdf.start == startPos]) > 0: #loop over this until it all start segments are removed from original way

        start = startPos
        end = nodepairdf[nodepairdf.start == startPos].values[0][1]

        for i in range(start, end): # delete nodes in orig way from startPos to end-1
            nodeID = nodelist[i]
            try: originalway.remove(originalway.find("./nd/[@ref='"+nodeID+"']"))
            except: pass
            if verbose == 'yes': print('removed node index',i)

        nodepairdf = nodepairdf.drop(nodepairdf[nodepairdf.start == startPos].index[0])  # delete this row from the df
        startPos = end


    # second, delete nodes from segements at the end of the way
    while len(nodepairdf[nodepairdf.end == endPos]) > 0:  
        end = endPos
        start = nodepairdf[nodepairdf.end == endPos].values[0][0]

        for i in range(start+1, end+1): # delete nodes in orig way from startPos +1 to endPos
            nodeID = nodelist[i]
            try: originalway.remove(originalway.find("./nd/[@ref='"+nodeID+"']"))
            except: pass
            if verbose == 'yes': print('removed node index',i)

        nodepairdf = nodepairdf.drop(nodepairdf[nodepairdf.end == endPos].index[0]) # delete this row from the df
        endPos = start



    copypairs =[]
    sortedNodepairdf = nodepairdf.sort_values(by=['start']).values

    for i, pair in enumerate(sortedNodepairdf): # the remaining rows should only be nodes of middle segments

        if pair[1] < pair[0]: continue # ignore reverse pairs
        if len(copypairs) == 0: 
            copypairs.append((startPos,pair[0]))
            lastEnd = pair[1]
        if i == len(sortedNodepairdf)-1: copypairs.append((pair[1],endPos))
        if pair[0] > lastEnd: copypairs.append((lastEnd,pair[0]))
        if i == len(remaining)-1 and pair[1] < endPos: copypairs.append((pair[1],endPos))

        for i in range(pair[0]+1, pair[1]): # delete nodes in orig way from startPos +1 to endPos
            nodeID = nodelist[i]
            try: originalway.remove(originalway.find("./nd/[@ref='"+nodeID+"']"))
            except: pass
            if verbose == 'yes': print('removed node index',i)

        lastEnd = pair[1]

    if verbose == 'yes': print('copy pairs:',copypairs)
    # create copynodes with no realspeed for all pairs in copypairs 
    for pair in copypairs:
        nodesequence = []
        for i in range(pair[0],pair[1]+1): nodesequence.append(nodelist[i])
        CopyNodes(originalway, nodesequence,'no','mph', str(newwayID_seed))
        newwayID_seed += 1


In [319]:

data_items = nodepairdict.items()
data_list = list(data_items)
nodepairdf = pd.DataFrame(data_list,columns=['start','end'])
print(nodepairdf.sort_values(by=['start']))

   start  end
1      1    2
0      2    3
2      3    2


In [323]:
nodelist

['9497668621', '4174213866', '4174213828', '53041273']

In [322]:
nodepairtuples

[(2, 1), (2, 3), (1, 2), (3, 2)]

In [257]:
nodepairdict[0] = 1
nodepairdict[1]= 3
nodepairdict[4] = 7
nodepairdict[8] = 9

In [258]:
nodepairdf

,start,end
2,4,7


In [263]:
## idea 3 for removing nodes



removed node index 0
removed node index 1
removed node index 2
removed node index 9
removed node index 5
removed node index 6
[(3, 4), (7, 8)]


In [375]:
newwayID_seed

12212

In [382]:
wayeg = root.find(".way/[@id = '" + '12206'+ "']")
ET.dump(wayeg)

<way id="12206" version="13" timestamp="2020-02-21T17:38:51Z" changeset="81327534" uid="119881" user="clay_c" synthetic="yes"><nd ref="65332847" /><nd ref="6914129193" /><nd ref="65361159" /><tag k="cycleway" v="shared_lane" />
		<tag k="highway" v="residential" />
		<tag k="lcn_ref" v="10" />
		<tag k="name" v="Pacific Avenue" />
		<tag k="tiger:cfcc" v="A41" />
		<tag k="tiger:county" v="San Francisco, CA" />
		<tag k="tiger:name_base" v="Pacific" />
		<tag k="tiger:name_type" v="Ave" />
	<tag k="maxspeed" v="18.869" /></way>


In [369]:
wayeg = root.find(".way/[@id = '" + str(newwayID_seed-1)+ "']")
ET.dump(wayeg)

AttributeError: 'NoneType' object has no attribute 'iter'

In [231]:
ET.dump(wayeg)

<way id="404588382" version="2" timestamp="2016-03-20T02:58:13Z" changeset="37949082" uid="933797" user="oba510">
		<nd ref="4068621437" />
		<nd ref="4068621431" />
		<nd ref="4068621451" />
		<nd ref="4068621423" />
		<nd ref="4068621430" />
		<nd ref="4068621455" />
		<nd ref="4068621460" />
		<nd ref="4068621472" />
		<nd ref="4068621426" />
		<tag k="highway" v="service" />
		<tag k="service" v="parking_aisle" />
	</way>
	


In [209]:
root.findall('./way/[@id="29480947"]')
# root.findway('./way/[@id ="476005551"]')

[<Element 'way' at 0x7fb6e478e130>]

In [165]:
newway = ET.Element('way')
for i,anode in enumerate(nodepair):    
    NodeRef = ET.Element('nd', attrib = {'ref':anode})
    newway.append(NodeRef)
    print(i)
    if i == len(nodepair)-1: continue
    try:
        originalway.remove(originalway.find("./nd/[@ref='"+anode+"']"))
    except: continue

0
1
2
3
4


In [325]:
sameway
# wayID, list(speedsDFpeak.loc[(speedsDFpeak.osm_start_node_id == startnode) & (speedsDFpeak.osm_end_node_id == endnode)]['osm_way_id'])[0]
root.findall("./way/[@id='" + wayID + "']")

[<Element 'way' at 0x7fc71ca31f40>]

In [326]:
wayID

'1030325062'

In [276]:
for way in root.find("./way/[@id ='11001']"):
    ET.dump(way)

<nd ref="65304103" />
<nd ref="65306688" />
<tag k="highway" v="residential" />
		
<tag k="name" v="29th Street" />
		
<tag k="tiger:cfcc" v="A41" />
		
<tag k="tiger:county" v="San Francisco, CA" />
		
<tag k="tiger:name_base" v="29th" />
		
<tag k="tiger:name_type" v="St" />
		
<tag k="tiger:reviewed" v="no" />
	
<tag k="maxspeed" v="17.294" />


In [49]:
# nodelist = []
# nodelistfloat = []
# for node in root.find("./way/[@id = '26965920']").iter('nd'):
#     ref = node.attrib['ref']
#     nodelist.append(ref)
#     nodelistfloat.append(int(ref))

speedsDFpeak.loc[(speedsDFpeak.osm_start_node_id.isin(nodelistfloat)) & (speedsDFpeak.osm_end_node_id.isin(nodelistfloat))]
# nodelistfloat = []
# for node in nodelist:
#     nodelistfloat.append(int(node))
# nodelistfloat


,Unnamed: 0,year,quarter,hour_of_day,segment_id,start_junction_id,end_junction_id,osm_way_id,osm_start_node_id,osm_end_node_id,speed_mph_mean,speed_mph_stddev,speed_mph_p50,speed_mph_p85
43666,867601,2020,1,8,c8b1bb5ab138296327e398f80cb1a5b1c7db1e0a,d2f4724929b90e5db871c75cab165930f99112a1,2d76f2ba50a0245fb10c4d2582e6dcd1bcff221f,26965920,65328428,4023031571,19.202,4.105,19.432,22.278
143253,2847866,2020,1,8,940ee79671cb92e930027ccf3413d9220cbff440,2d76f2ba50a0245fb10c4d2582e6dcd1bcff221f,a46733aa745de8ce11630b6fa51b09e28f76c3d9,26965920,4023031571,65354844,18.346,2.977,18.373,21.046
169732,3374599,2020,1,8,a576339a2119f0c0b835318aba8047b9cde2d88f,a46733aa745de8ce11630b6fa51b09e28f76c3d9,d2f4724929b90e5db871c75cab165930f99112a1,399575097,65354844,65328428,15.911,3.511,16.414,18.465


In [327]:
relevantSpeeds

,Unnamed: 0,year,quarter,hour_of_day,segment_id,start_junction_id,end_junction_id,osm_way_id,osm_start_node_id,osm_end_node_id,speed_mph_mean,speed_mph_stddev,speed_mph_p50,speed_mph_p85
31889,633690,2020,1,8,d1814dbd3591ec714a8d3e89b155f3e31197bd0a,53342024cf2ac7020c0af07f3940160a0b096a1e,cab2f5801a962745908f9f76df435cf1eba0a4b2,416147493,4174213828,4174213866,21.868,4.964,22.050,26.811
180374,3586278,2020,1,8,9704242fd731a2cb6588d071f5aaee0f7e6655ef,53342024cf2ac7020c0af07f3940160a0b096a1e,052ff188e278708a3562ebd1567f52183b1d84b0,416147493,4174213828,53041273,11.897,8.585,13.035,20.650
201004,3996256,2020,1,8,d1814dbd3591ec714a8d3e89b155f3e31197bd0a,cab2f5801a962745908f9f76df435cf1eba0a4b2,53342024cf2ac7020c0af07f3940160a0b096a1e,416147493,4174213866,4174213828,10.871,9.037,9.160,21.151
201005,3996272,2020,1,8,9704242fd731a2cb6588d071f5aaee0f7e6655ef,052ff188e278708a3562ebd1567f52183b1d84b0,53342024cf2ac7020c0af07f3940160a0b096a1e,416147493,53041273,4174213828,19.559,5.299,19.428,24.575


In [429]:
progress = 100000/(number_ofways-1)
int(progress*100)

60

In [ ]:
## Idea 1 for removing nodes

sorted(nodepairtuples)
startPos = 0
endPos = len(nodelist)-1

for pair in sorted(nodepairtuples):
    if  pair[0] == startPos and pair[1] == endPos:
        # check if reverse exists, if yes delete all nodes, 
        # delete all nodes
        break
    elif pair[0] == startPos: # case where nodepair starts at beginning but not end
        if pair[1]>pair[0] and (pair[1],pair[0]) in nodepairtuples: # case where nodepair is in orig direction
        # delete nodes from startPos to pair[1]-1
       
        startPos = pair[1]
        
    elif pair[1]== endPos:
        # delete nodes from startPos +1 to endPos
        endPos = pair[0]
#     else: # case where pair is in the middle of nodelist
        #delete any nodes between startPos and endPos. from and including startPos+1 to endPos-1
        # create split with  new ways no speed [startPos:pair[0]] and  [pair[1]:endPos]
        
# copyNodes (without speed) if there are jumps in the indexes. E.g. [0,1,2,3,5,6,7] -> [0,1,2,3] and [5,6,7] as separate ways
# e.g [0,1,2,4,5,7,8] -> [0,1,2], 


In [ ]:
## Idea 2 for removing nodes

sorted(nodepairtuples)
startPos = 0
endPos = len(nodelist)-1
nodepairtuplescopy = nodepairtuples.copy()



for pair in nodepairtuples:
    if pair[0] == startPos:
        # delete nodes in orig way from startPos to pair[1]-1
        startPos = pair[1]
        del nodepairtuplescopy[nodepairtuplescopy.index(pair)]
    elif pair[1] == endPos:
        # delete nodes in orig way from startPos +1 to endPos
        endPos = pair[0]
        del nodepairtuplescopy[nodepairtuplescopy.index(pair)]
    else: 

Questions to be checked:
- are there real speeds for nodes that are not right ater one another? can there be a list of nodes > 2 with real speed available for the whol elist (displayed as node1 - lastnode?
    yes. there are, take the entire nodelist
- deal with an empty nodelist
- deal with not being able to overwrite an existing way with ID, instead it adds more of the same wayID elements
- deal with the remove not working
    done
- is it an issue if the nd ref eelents are at the end?
    appending them to the beginnging now so no needs to test this
- how to create a new way ID counter system in the global var (not hard)
- deal with mapbox which doesnt have wayID
    switched to searching with nodes
- add column to speedDF and indicate speed was used
    no need since node search switch
- check if pandas can find nodelist as both start node and end node
    yes, done
- add optional parameter to add one way tag
     - removed one way tag from function
- add check if osm way ID is the same for the reverse speed pair
    - added for uber only ot for mapbox. ran into issue where the osm wayID might not match uber speed way ID for same way (uber outdated)
- consider not adding reverse pairs to the dict for removing nodes
    - yes, no longer adding reverse pairs for the removal. also not adding one way tags. some service roads not tagged one way are highway exits which are de facto one way in a routing engine if highway is one way
    
Issue with removing nodes from the original way

- first choice is that removal will be based only on the correct direction, not the reverse
    i am trying to incoroprate direction in the new ways and deleting function
    - possible solution: no adding one way tags to new ways. if way has oneway tag, it will be copied over, if it does not, we do not add one even if we create a new way and its reverse, we let the node sequnce determine routing direction. 
    
- possible issue: if the set of nodes belongs to more than one way: like 2 one way ways in opposite directions.
right now, code does not query the osm network to see where the nodes belong
    - added a check if the osm way ID is different in the uber speed df, then it does not take the speed for the reverse nodes and differnet way. if it is indeed a diff way, then it will get its turn over when interating over all ways in the osm
    - for mapbox since there is no wayID in the speed data, the check will be if the node pair exists in more than one way, then the speed for the reverse nodes will not be taken

